In [39]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#XlsxWriter: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import xlsxwriter # importando a biblioteca xlsxwriter

#Locale: Possui uma série de funções e comandos para trabalharmos com formatação de moeda
import locale # importando a biblioteca locale

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8') # definindo o locale para pt_BR

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Importação das Planilhas

### ABA 1

In [40]:
pasta = 'BASE'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. Fila na UF'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo = os.path.splitext(arquivo)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo  # Adiciona uma coluna 'UF' com o nome do arquivo
        dfs_dict[nome_arquivo] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba1 = pd.concat(dfs_dict.values(), ignore_index=True)


In [41]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'CÓDIGO DO PROCEDIMENTO NO SIGTAP','Unnamed: 1':'PROCEDIMENTO CIRÚRGICO',
                                 'Unnamed: 2':'QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22', 'Unnamed: 3':'Redução do Tamanho da Fila (%)', 'Unnamed: 4':'Prazo para reduzir o % proposto (em meses)', 
                                 'Unnamed: 5':'Qtde de cirurgias a serem feitas no prazo pactuado', 'Unnamed: 6':'SQ (CODIGO Interno', 'UF':'UF'} , inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba1.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1.dropna(subset=['PROCEDIMENTO CIRÚRGICO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1.drop(df_planilha_aba1[df_planilha_aba1['PROCEDIMENTO CIRÚRGICO'] == 'TOTAL'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1.drop(df_planilha_aba1[df_planilha_aba1['PROCEDIMENTO CIRÚRGICO'] == 'PROCEDIMENTO CIRÚRGICO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'] = df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'].astype(int) # Converte a coluna 'Qtde de cirurgias a serem feitas no prazo pactuado' para inteiro

In [42]:
df_planilha_aba1.head(10)

,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pactuado,SQ (CODIGO Interno,UF
2,401020029,ENXERTO DERMO-EPIDERMICO,9,0,NaN,0,100001,ACRE
3,401020037,ENXERTO LIVRE DE PELE TOTAL,14,0,NaN,0,100002,ACRE
4,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",20,0,NaN,0,100003,ACRE
5,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,17,0,NaN,0,100004,ACRE
6,401020061,EXERESE DE CISTO BRANQUIAL,28,0,NaN,0,100005,ACRE
7,401020070,EXERESE DE CISTO DERMOIDE,138,0,NaN,0,100006,ACRE
8,401020088,EXERESE DE CISTO SACRO-COCCIGEO,1,0,NaN,0,100007,ACRE
9,401020096,EXERESE DE CISTO TIREOGLOSSO,18,1,9,18,100008,ACRE
10,401020100,EXTIRPACAO E SUPRESSAO DE LESAO DE PELE E DE T...,10,0,NaN,0,100009,ACRE
11,401020150,TRATAMENTO CIRURGICO DO SINUS PRE-AURICULAR,8,0,NaN,0,100010,ACRE


### ABA 2

In [43]:
pasta = 'BASE'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. CNES e Proced.'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo = os.path.splitext(arquivo)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo  # Adiciona uma coluna 'UF' com o nome do arquivo
        dfs_dict[nome_arquivo] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba2 = pd.concat(dfs_dict.values(), ignore_index=True)


In [44]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP', 
                                 'Unnamed: 3':'PROCEDIMENTO CIRÚRGICO (PRINCIPAL)', 'Unnamed: 4':'COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual)', 'Unnamed: 5':'GESTÃO DO SERVIÇO',
                                 'Unnamed: 6':'CODIGO DA NATUREZA JURÍDICA','Unnamed: 7':'NATUREZA JURÍDICA', 'Unnamed: 8':'POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?',
                                 'Unnamed: 9':'Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B)','Unnamed: 10':'SQ (CODIGO Interno'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba2.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba2.dropna(subset=['ESTABELECIMENTO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba2.drop(df_planilha_aba2[df_planilha_aba2['ESTABELECIMENTO'] == 'ESTABELECIMENTO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'


In [45]:
df_planilha_aba2.head(10)


,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual),GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B),SQ (CODIGO Interno,UF
2,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200001,ACRE
3,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060186,LAQUEADURA TUBARIA,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200002,ACRE
4,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060046,CURETAGEM SEMIOTICA C/ OU S/ DILATACAO DO COLO...,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200003,ACRE
5,2001586,FUNDHACRE,409010189,LITOTRIPSIA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200004,ACRE
6,2001586,FUNDHACRE,409010200,NEFRECTOMIA PARCIAL,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200005,ACRE
7,2001586,FUNDHACRE,409010367,RESSECCAO DO COLO VESICAL / TUMOR VESICAL A CE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200006,ACRE
8,2001586,FUNDHACRE,409020176,URETROTOMIA INTERNA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200007,ACRE
9,2001586,FUNDHACRE,409010170,INSTALACAO ENDOSCOPICA DE CATETER DUPLO J,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200008,ACRE
10,2001586,FUNDHACRE,406020078,IMPLANTAÇÃO DE CATETER DE LONGA PERMANÊNCIA SE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200009,ACRE
13,2001586,FUNDHACRE,409070050,COLPOPERINEOPLASTIA ANTERIOR E POSTERIOR,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200012,ACRE


### ABA 3

In [46]:
pasta = 'BASE'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Execução'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo = os.path.splitext(arquivo)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo  # Adiciona uma coluna 'UF' com o nome do arquivo
        dfs_dict[nome_arquivo] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba3 = pd.concat(dfs_dict.values(), ignore_index=True)

In [47]:
df_planilha_aba3.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'Gestão do Recurso','Unnamed: 2':'DESCRIÇÃO DO GESTOR','Unnamed: 3':'VALOR',
                                 'Unnamed: 4':'março','Unnamed: 5':'abril','Unnamed: 6':'maio','Unnamed: 7':'junho','Unnamed: 8':'julho','Unnamed: 9':'agosto','Unnamed: 10':'setembro','Unnamed: 11':'outubro',
                                 'Unnamed: 12':'novembro','Unnamed: 13':'dezembro','Unnamed: 14':'% TOTAL','SQ (CODIGO Interno':'SQ (CODIGO Interno'}, inplace=True)

df_planilha_aba3.drop(df_planilha_aba3.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3.dropna(subset=['CODIGO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.drop(df_planilha_aba3[df_planilha_aba3['CODIGO GESTOR'] == 'CODIGO GESTOR'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.dropna(subset=['% TOTAL'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.dropna(subset=['DESCRIÇÃO DO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].map(lambda x: locale.currency(float(x), grouping=True)) # Formata a coluna 'VALOR' para moeda

cols = ["março", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro", "% TOTAL"] # Define as colunas que serão formatadas
df_planilha_aba3[cols] = df_planilha_aba3[cols].applymap(lambda x: '{:.0%}'.format(float(x))) # Formata as colunas para porcentagem
df_planilha_aba3.head(10) # Exibe as 5 primeiras linhas do arquivo


,CODIGO GESTOR,Gestão do Recurso,DESCRIÇÃO DO GESTOR,VALOR,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,% TOTAL,SQ (CODIGO Interno,UF
6,120000,ESTADUAL,Acre,"R$ 2.550.776,40",nan%,10%,15%,15%,10%,10%,10%,10%,10%,10%,100%,30001,ACRE
1026,530000,ESTADUAL,Distrito Federal,"R$ 8.703.429,35",5%,10%,11%,11%,11%,11%,11%,11%,11%,8%,100%,30019,DISTRITO FEDERAL
2010,210000,ESTADUAL,Maranhão,"R$ 13.001.793,26",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30001,MARANHÃO
2011,210005,MUNICIPAL,ACAILANDIA,"R$ 107.553,10",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30002,MARANHÃO
2012,210060,MUNICIPAL,AMARANTE DO MARANHAO,"R$ 211.279,69",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30003,MARANHÃO
2013,210140,MUNICIPAL,BALSAS,"R$ 337.372,49",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30004,MARANHÃO
2014,210160,MUNICIPAL,BARRA DO CORDA,"R$ 203.398,03",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30005,MARANHÃO
2015,210230,MUNICIPAL,BURITI BRAVO,"R$ 22.982,22",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30006,MARANHÃO
2016,210232,MUNICIPAL,BURITICUPU,"R$ 17.448,57",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30007,MARANHÃO
2017,210280,MUNICIPAL,CAROLINA,"R$ 42.741,91",4%,14%,15%,10%,12%,10%,10%,10%,10%,5%,100%,30008,MARANHÃO


## SALVAR ARQUIVO

### EXCEL

In [48]:
writer = pd.ExcelWriter(f'PLANILHA/BASE_G.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1.to_excel(writer, sheet_name='Ident. Fila na UF', index=False)
df_planilha_aba2.to_excel(writer, sheet_name='Ident. CNES e Proced.', index=False)
df_planilha_aba3.to_excel(writer, sheet_name='Execução', index=False)

writer.save()

### CSV

In [49]:
df_planilha_aba1.to_csv('PLANILHA/BASE_G_1.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba2.to_csv('PLANILHA/BASE_G_2.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba3.to_csv('PLANILHA/BASE_G_3.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv

In [50]:
tempo_final = time.time()
tempo_total = (float(tempo_final - tempo_inicial)/60)
print(f"Tempo total de execução: {tempo_total:,.1f} minutos" )

Tempo total de execução: 0.4 minutos


: 